<a href="https://colab.research.google.com/github/tranquil-morning/CA-deeplearning/blob/main/07_LSTM_Music_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
text = open("drive/MyDrive/pianoabc.txt", "r").read()
unique_text = list(set(text))
unique_text.sort()
unique_text

['"',
 "'",
 ',',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 '^',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'z']

In [4]:
text_to_num = {}
num_to_text = {}
for i, j in enumerate(unique_text):
  text_to_num[j] = i
  num_to_text[i] = j

text_to_num

{'"': 0,
 "'": 1,
 ',': 2,
 '/': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 ':': 13,
 'A': 14,
 'B': 15,
 'C': 16,
 'D': 17,
 'E': 18,
 'F': 19,
 'G': 20,
 '^': 21,
 '_': 22,
 'a': 23,
 'b': 24,
 'c': 25,
 'd': 26,
 'e': 27,
 'f': 28,
 'g': 29,
 'z': 30}

In [5]:
numberized_text = []

for i in text:
  numberized_text.append(text_to_num[i])

In [6]:
X = []
Y = []

for i in range(0, len(numberized_text) - 25):
  X.append(numberized_text[i:i+25])
  Y.append(numberized_text[i + 25])

In [7]:
import tensorflow as tf

X = tf.one_hot(X, 31)
Y = tf.one_hot(Y, 31)

In [8]:
X[0]
Y[0]

<tf.Tensor: shape=(31,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

In [9]:
X[0].shape

TensorShape([25, 31])

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(100, input_shape = (25, 31)),
  tf.keras.layers.Dense(31, activation = "softmax")
])

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"]) # use spare categorical in no one_hot encode

model.fit(X, Y, batch_size = 64, epochs = 100, verbose = 2)

model.save("drive/MyDrive/model1")

In [11]:
Pmodel = tf.keras.models.load_model("drive/MyDrive/model1")

TensorShape([1, 25, 31])

In [17]:
f_input = numberized_text[117: 117+25]
f_input = tf.one_hot(f_input, 31)
f_input = tf.expand_dims(f_input, axis = 0)

f_input.shape

TensorShape([1, 25, 31])

In [19]:
import numpy as np
f_pred = Pmodel.predict(f_input)

print(num_to_text[np.argmax(f_pred[0])])
print(numberized_text[117 + 25])


2
5


In [ ]:
music = []

input = numberized_text[117: 117+25]
input = tf.one_hot(input, 31)
input = tf.expand_dims(input, axis = 0)


for i in range(200):

  pred = Pmodel.predict(input)
  pred = np.argmax(pred[0])
  music.append(pred)

  n_input = input.numpy()[0][1: ]
  one_hot_pred = tf.one_hot(pred, 31).numpy()
  n_input = np.vstack([n_input, one_hot_pred])
  input = n_input
  input = tf.expand_dims(input, axis = 0)


music




In [39]:
music_text = []
for i in music:
  music_text.append(num_to_text[i])

print("".join(music_text))

2a/2g/2f"A7"e/2d/2c/2A/2"D"dfd/2e/2f/2g/2"D"a/2a/2b/2a/2"E7"g/2f/2e/2g/2"A"aa"D"a/2b/2a/2f/2"A"e/2c/2A/2c/2"D"d/2d/2d/2e/2"D"f/2e/2d/2c/2"G"d/2B/2c/2d/2"A"e/2d/2c/2B/2"D"A/2B/2A/2F/2"A7"G/2E/2E/2G/2"D


https://www.abcjs.net/abcjs-editor.html